<h1>OVERVIEW</h1>
<p>This section describe the overall understanding of the dataset, <br/>
    provide evidence of structured data, and the class destribution. <br/>Then attempt to cartegorise the dataset based on datatype, finaly balance the dataset using smote </p>

In [13]:
# pip install --upgrade tensorflow
%matplotlib inline
import warnings
# ignore all warnings
warnings.filterwarnings('ignore')

In [14]:
# pip install altair

In [15]:
# pip install imblearn

In [16]:
# pip install pydot

In [17]:
# pip install matplotlib

In [18]:
import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE

In [19]:
# this function breakes the dataset in three section
# int,float and string
def feature_target_split(data):
  label = data.pop('attack_cat')
  return data,label
def group_by_dtype(data):
    columns=data.columns # get list of columns
    unique=data.dtypes.unique() # select one of each dtype
    result={'int':[],'float':[],'object':[],'labels':[]}
    for col in columns:
      if data[col].dtypes=='float64': 
        temp= result['float'] # asign float64 array
      elif data[col].dtypes=='int64':
        temp= result['int'] # asign int array
      else:
        temp= result['object'] # asign object array
      temp.append(col) # append the array
    return result
# this function reamples the dataset using the smote imbalance 
# from imbalace
def balance_dataset(X,y):
  sm = SMOTE(random_state=42)
  X_res, y_res = sm.fit_resample(X, y)
  return X_res, y_res

In [20]:
# importing the train and test data dataset from file
imbalanced_train_val = pd.read_csv('./data/UNSW_NB15_training-set.csv')
test = pd.read_csv('./data/UNSW_NB15_testing-set.csv')
# Drop unwanted  columns such as ID and label for all data
# imported datasets
imbalanced_train_val=imbalanced_train_val.drop(columns=['id','label'])
test=test.drop(columns=['id','label'])
# combine the dataset for overall dataset understanding
combined_df= pd.concat([imbalanced_train_val,test])
print(combined_df.shape, 'combined examples')
print(imbalanced_train_val.shape, 'imbalanced_train_val examples')
print(test.shape, 'test examples')

(257673, 43) combined examples
(82332, 43) imbalanced_train_val examples
(175341, 43) test examples


In [21]:
# dusplay of the training dataset to identify the features
df_training = pd.read_csv('./data/UNSW_NB15_training-set.csv')
df_training.head()

,id,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,...,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat,label
0,1,0.000011,udp,-,INT,2,0,496,0,90909.0902,...,1,2,0,0,0,1,2,0,Normal,0
1,2,0.000008,udp,-,INT,2,0,1762,0,125000.0003,...,1,2,0,0,0,1,2,0,Normal,0
2,3,0.000005,udp,-,INT,2,0,1068,0,200000.0051,...,1,3,0,0,0,1,3,0,Normal,0
3,4,0.000006,udp,-,INT,2,0,900,0,166666.6608,...,1,3,0,0,0,2,3,0,Normal,0
4,5,0.000010,udp,-,INT,2,0,2126,0,100000.0025,...,1,3,0,0,0,2,3,0,Normal,0


In [22]:
# Display first 5 rows of the dataset
combined_df.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.000011,udp,-,INT,2,0,496,0,90909.0902,254,...,1,1,2,0,0,0,1,2,0,Normal
1,0.000008,udp,-,INT,2,0,1762,0,125000.0003,254,...,1,1,2,0,0,0,1,2,0,Normal
2,0.000005,udp,-,INT,2,0,1068,0,200000.0051,254,...,1,1,3,0,0,0,1,3,0,Normal
3,0.000006,udp,-,INT,2,0,900,0,166666.6608,254,...,2,1,3,0,0,0,2,3,0,Normal
4,0.000010,udp,-,INT,2,0,2126,0,100000.0025,254,...,2,1,3,0,0,0,2,3,0,Normal


In [23]:
#step 2: Check for empty cell using the ombined dataset
print('number of empty cell in combined set is: {}'.format(combined_df.isna().sum().sum()))

number of empty cell in combined set is: 0


In [24]:
_combined_df=combined_df.copy()
_features,_target=feature_target_split(_combined_df)
print('Feature shape: {} rows and {} columns'.format(_features.shape[0],_features.shape[1]))
print('Target shape: {} rows and 1 columns'.format(_target.shape[0]))


Feature shape: 257673 rows and 42 columns
Target shape: 257673 rows and 1 columns


In [25]:
# using the combined datadets to identify the datatypes with the function
# group_by_dtype
features_col=group_by_dtype(_features)
# # load a dataset
features_table=pd.DataFrame({'Name':['int','float','object'],'Count':[len(features_col['int']),len(features_col['float']),len(features_col['object'])]})
# plot the dataset, referencing dataframe column names
alt.Chart(features_table, title="Categories of dataframe columns by type").mark_bar().encode(
  x='Count',
  y='Name',
  color='Name'
)

alt.Chart(...)

In [26]:
# imbalanced data
# Visualise the target categories in the train_val dataset
combined=combined_df.copy()
_features,_target=feature_target_split(combined)
target_table=_target.value_counts().rename_axis('Name').reset_index(name='Count')
alt.Chart(target_table, title="Categories of target column in imbalanced dataset (attack type)").mark_bar().encode(
  x='Count',
  y='Name',
  color='Name'
)

alt.Chart(...)

In [27]:
# for the purpose of training and validation of the dataset
# the trainng dataset is labeled train-val and in this section
# id was splited into different cartegory based on data type 
# using the feature_target_split function defined above
train_val_df=imbalanced_train_val.copy()
_features,_target=feature_target_split(train_val_df)

In [28]:
# # create balansed dataset from previous dataset using the 
# fuction defined above
# however using smote imalance data generates an error
# so the solution was to
# 1: define a variable to hold all dtype features in this case
# 3 types as defined cell 13 above 
_uniques={}
#2: seperate the string features from the remening feature columne
string_features = features_col['object']
# initialise the datafraime thasat holdes all the feture dataset
# on both input and result
_df=_features.copy()
encoded_df=_features.copy()
# string_df are dataset containing the string type
string_df=_df[string_features]
#3: itterate on the string_features defined above to convert all
# dataset of type object to number using factorize.
for n in string_features:
    codes, uniques = pd.factorize(string_df[n])
    _uniques[n]=uniques
    string_df[n]=codes
# replace the type objet to the factorise data
encoded_df[string_features]=string_df
encoded_df

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports
0,0.000011,0,0,0,2,0,496,0,90909.090200,254,...,1,1,1,2,0,0,0,1,2,0
1,0.000008,0,0,0,2,0,1762,0,125000.000300,254,...,1,1,1,2,0,0,0,1,2,0
2,0.000005,0,0,0,2,0,1068,0,200000.005100,254,...,1,1,1,3,0,0,0,1,3,0
3,0.000006,0,0,0,2,0,900,0,166666.660800,254,...,2,2,1,3,0,0,0,2,3,0
4,0.000010,0,0,0,2,0,2126,0,100000.002500,254,...,2,2,1,3,0,0,0,2,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000005,0,0,0,2,0,104,0,200000.005100,254,...,2,1,1,2,0,0,0,2,1,0
82328,1.106101,2,0,1,20,8,18062,354,24.410067,254,...,2,1,1,1,0,0,0,3,2,0
82329,0.000000,1,0,0,1,0,46,0,0.000000,0,...,1,1,1,1,0,0,0,1,1,1
82330,0.000000,1,0,0,1,0,46,0,0.000000,0,...,1,1,1,1,0,0,0,1,1,1


In [29]:
# based on the above
# all dataset has been converted in to integer and float
# and can be resampled now
print('Original dataset shape %s' % Counter(_target))
_features_res,_target_res=balance_dataset(encoded_df,_target)
print('Resampled dataset shape %s' % Counter(_target_res))

Original dataset shape Counter({'Normal': 37000, 'Generic': 18871, 'Exploits': 11132, 'Fuzzers': 6062, 'DoS': 4089, 'Reconnaissance': 3496, 'Analysis': 677, 'Backdoor': 583, 'Shellcode': 378, 'Worms': 44})
Resampled dataset shape Counter({'Normal': 37000, 'Reconnaissance': 37000, 'Backdoor': 37000, 'DoS': 37000, 'Exploits': 37000, 'Analysis': 37000, 'Fuzzers': 37000, 'Worms': 37000, 'Shellcode': 37000, 'Generic': 37000})


In [30]:
# after the resampling
# the dataset is then replased back to the initail 
# string value
features_res=_features_res.copy()
for n in string_features:
    uniques=_uniques[n]
    features_res[n]=features_res[n].apply(lambda x: uniques[x])
resampled_train_val = pd.concat([features_res, _target_res], axis=1) # train val dataset resampled

In [31]:
# the resampled data is now back to is original 
# data type caount
resampled_train_val

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,...,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,is_ftp_login,ct_ftp_cmd,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,is_sm_ips_ports,attack_cat
0,0.000011,udp,-,INT,2,0,496,0,90909.090200,254,...,1,1,2,0,0,0,1,2,0,Normal
1,0.000008,udp,-,INT,2,0,1762,0,125000.000300,254,...,1,1,2,0,0,0,1,2,0,Normal
2,0.000005,udp,-,INT,2,0,1068,0,200000.005100,254,...,1,1,3,0,0,0,1,3,0,Normal
3,0.000006,udp,-,INT,2,0,900,0,166666.660800,254,...,2,1,3,0,0,0,2,3,0,Normal
4,0.000010,udp,-,INT,2,0,2126,0,100000.002500,254,...,2,1,3,0,0,0,2,3,0,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369995,1.424299,tcp,http,FIN,15,45,1432,50553,33.565271,254,...,1,1,1,0,0,0,1,1,0,Worms
369996,0.467546,tcp,http,FIN,10,6,1298,268,46.072385,254,...,1,1,1,0,0,1,1,1,0,Worms
369997,3.518716,tcp,http,FIN,41,220,2353,280081,54.888184,254,...,2,1,1,0,0,1,2,1,0,Worms
369998,0.525146,tcp,http,FIN,10,6,1267,268,30.279165,254,...,1,1,1,0,0,1,1,1,0,Worms


In [32]:
# evidence of resampled data
resampled=resampled_train_val.copy()
_features,_target=feature_target_split(resampled)
target_table=_target.value_counts().rename_axis('Name').reset_index(name='Count')
alt.Chart(target_table, title="Categories of target column in train val (attack type)").mark_bar().encode(
  x='Count',
  y='Name',
  color='Name'
)

alt.Chart(...)

In [33]:
# store that data in the current kernel
# this is done to avoid repeting the same code in the 
# subsequent juputer file
IMBALANCED_TRAIN_VAL =imbalanced_train_val
RESAMPLED_TRAIN_VAL =resampled_train_val
TEST=test
%store IMBALANCED_TRAIN_VAL
%store RESAMPLED_TRAIN_VAL
%store TEST

Stored 'IMBALANCED_TRAIN_VAL' (DataFrame)
Stored 'RESAMPLED_TRAIN_VAL' (DataFrame)
Stored 'TEST' (DataFrame)
